#  Data Munchers

Hotel reviews project.
Data: https://www.kaggle.com/jiashenliu/515k-hotel-reviews-data-in-europe

In [1]:
import pandas as pd

users = pd.read_csv('../Hotel_Reviews.csv')

users.head()

,Hotel_Address,Additional_Number_of_Scoring,Review_Date,Average_Score,Hotel_Name,Reviewer_Nationality,Negative_Review,Review_Total_Negative_Word_Counts,Total_Number_of_Reviews,Positive_Review,Review_Total_Positive_Word_Counts,Total_Number_of_Reviews_Reviewer_Has_Given,Reviewer_Score,Tags,days_since_review,lat,lng
0,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,8/3/2017,7.7,Hotel Arena,Russia,I am so angry that i made this post available...,397,1403,Only the park outside of the hotel was beauti...,11,7,2.9,"[' Leisure trip ', ' Couple ', ' Duplex Double...",0 days,52.360576,4.915968
1,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,8/3/2017,7.7,Hotel Arena,Ireland,No Negative,0,1403,No real complaints the hotel was great great ...,105,7,7.5,"[' Leisure trip ', ' Couple ', ' Duplex Double...",0 days,52.360576,4.915968
2,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,7/31/2017,7.7,Hotel Arena,Australia,Rooms are nice but for elderly a bit difficul...,42,1403,Location was good and staff were ok It is cut...,21,9,7.1,"[' Leisure trip ', ' Family with young childre...",3 days,52.360576,4.915968
3,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,7/31/2017,7.7,Hotel Arena,United Kingdom,My room was dirty and I was afraid to walk ba...,210,1403,Great location in nice surroundings the bar a...,26,1,3.8,"[' Leisure trip ', ' Solo traveler ', ' Duplex...",3 days,52.360576,4.915968
4,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,7/24/2017,7.7,Hotel Arena,New Zealand,You When I booked with your company on line y...,140,1403,Amazing location and building Romantic setting,8,3,6.7,"[' Leisure trip ', ' Couple ', ' Suite ', ' St...",10 days,52.360576,4.915968


In [2]:
#Creating new dataframe with reviews columns cobined into one as well as cleaning 'No Negative / Positive' entries.
import random

#remove all other columns
def dropCols(newDF):
    newDF['Reviewer_Score'].describe()
    newDF = newDF.drop(["Hotel_Address", "Additional_Number_of_Scoring", "Review_Date", "Average_Score", 
                        "Reviewer_Nationality", "Review_Total_Negative_Word_Counts", "Total_Number_of_Reviews",
                       "Review_Total_Positive_Word_Counts", "Total_Number_of_Reviews_Reviewer_Has_Given",
                       "Tags", "days_since_review", "lng", "lat", "Hotel_Name"], axis=1)
    return newDF

#reduce the size of the data to our liking
#It randomly chooses rows from the entire dataset to reduce the bias
#rng is the parameter of how many entries we want in our dataset
def reduceDataSize(newDF, rng):    
    nums = random.sample(range(0, 515737), rng)
    d = pd.DataFrame()
    for row in nums:
        d = d.append(newDF.iloc[row, :])
    return d
        
#fix the no negative and no positive entries
def removeNoNeg_NoPos(df):
    df["Negative_Review"] = df["Negative_Review"].replace('No Negative', '')
    df["Positive_Review"] = df["Positive_Review"].replace('No Positive', '')
    return df

#concatinate the positive and negative entries into one review entry
def mergePosAndNeg(newDF):
    newDF["Review"] = newDF["Negative_Review"].map(str) + newDF["Positive_Review"]
    newDF = newDF.drop(["Negative_Review", "Positive_Review"], axis = 1)
    return newDF

#classify review score as good or bad
def classifyScores(newDF):
    scores = []
    for i in newDF["Reviewer_Score"]:
        if i > 5:
            #strr = 'good ' + str(i)
            scores.append(1)
            #print(i)
        else:
            #strr = 'bad ' + str(i)
            scores.append(0)
    newDF['Reviewer_Score'] = scores
    return newDF
    
#newDF['Reviewer_Score_Class'] = scores
#newDF['Reviewer_Score'] = scores
#newDF

In [3]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

nltk.download('stopwords')
nltk.download('wordnet')
stop = stopwords.words('english')

def toLowerCase(newDF):
    newDF['Review'] = newDF['Review'].astype(str).str.lower()
    return newDF

def removeStopWords(text):
    new_text = ""
    word_list = text.split()
    for word in word_list:
        if word not in stop:
            new_text = new_text + word + ' '
    return new_text
        
def rootForm(text):
    new_text = ""
    word_list = text.split()
    for word in word_list:
        new_text = new_text + WordNetLemmatizer().lemmatize(word) + ' '
    return new_text

def removeNumbers(text):
    new_text = ""
    word_list = text.split()
    for word in word_list:
        if not any(c.isdigit() for c in word):
            new_text = new_text + word + ' '
    return new_text

def transform(newDF, how):
    newDF['Review'] = newDF['Review'].apply(lambda x: how(x))                               
    return newDF


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Judit\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Judit\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [13]:
df = dropCols(users) # drops all unused columns
df = reduceDataSize(df, 100) # reduce the size of the data to our liking. 10 is just an arbitrary number of rows we want
df = removeNoNeg_NoPos(df) # removes the 'no positive' and 'no negative' entries
df = mergePosAndNeg(df) # if we dont want to merge we can just comment this line out
df = classifyScores(df) # classifies the reviews as either 'good' or 'bad'. Can be 1 or 0. Threshhold is 5
df["Origin"] = df["Review"]
df

,Reviewer_Score,Review,Origin
57037,1,Cookie on arrival balloons and cake in our ro...,Cookie on arrival balloons and cake in our ro...
317975,0,Twin beds when requested double and booked 9 ...,Twin beds when requested double and booked 9 ...
121659,1,Location far away to many famous place howeve...,Location far away to many famous place howeve...
320782,1,Excellent location Helpfull and feendy staff,Excellent location Helpfull and feendy staff
260766,1,the room was a little bit noisy with the nois...,the room was a little bit noisy with the nois...
410507,1,breakfast options for Muslims need to be more...,breakfast options for Muslims need to be more...
404082,1,Pillows Breakfast was very varied and excellent,Pillows Breakfast was very varied and excellent
505210,1,Hotel was fantastic but on arrival was told t...,Hotel was fantastic but on arrival was told t...
475689,1,During Expo in Milan accomodation prices have...,During Expo in Milan accomodation prices have...
95385,1,Our room was TINY We had absolutely no room t...,Our room was TINY We had absolutely no room t...


In [15]:
df = toLowerCase(df) # lower reviews
df = transform(df, removeStopWords) # remove useless words like ‘the’, ‘a’ ,’this’ etc.
df = transform(df, rootForm) # transform every word into their root form
df = transform(df, removeNumbers) # remove numbers

In [17]:
from sklearn.feature_extraction.text import CountVectorizer
reviews = df['Review']
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(reviews)
feature_names = vectorizer.get_feature_names()
#print(feature_names)
X = X.toarray()
X = X.transpose()
X.shape

mx_doc = {}
for num, col in enumerate(feature_names):
    mx_doc[col] = X[num]
#mx_doc
df_final = pd.DataFrame(data= mx_doc)
df_final['Reviewer_Score'] = df['Reviewer_Score'].values

In [18]:
#Save the cleaned data to another csv we can use for our models
df_final.to_csv("../Data/clean_data.csv", index=False, encoding='utf8')

#Save the cleaned data to another csv we can use for our models
df.to_csv("../Data/simple_clean_data.csv", index=False, encoding='utf8')